In [1]:
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact, ranksums, wilcoxon
import statsmodels.stats.multitest as multi

In [2]:
data = pd.read_csv('Demo_dataset.tsv', sep='\t', index_col=0)

In [5]:
statistics = []
control_samples = [i for i in data.columns if 'Mild' in i]
disease_samples = [i for i in data.columns if 'Severe' in i]
for item in data.index:
    control_exp = data.loc[item, control_samples]
    disease_exp = data.loc[item, disease_samples]
    try:
        f, p_value = ranksums(disease_exp, control_exp) # wilcoxon 成组非参检验 / ranksums 秩和检验
    except:
        f, p_value = 0, 1.0
    FC = disease_exp.mean()/control_exp.mean() if control_exp.mean()!=0 else np.inf
    statistics.append([item, data.loc[item, 'KO'], len(control_samples), len(disease_samples), 
                       control_exp.mean(), disease_exp.mean(), FC, p_value])
statistics = pd.DataFrame(statistics, index = data.index, columns = ['ID', 'KO', 'N_control', 'N_disease', 
                                                                       'AveExpr_control', 'AveExpr_disease', 
                                                                       'FC','P.Value'])
statistics['adj.P.Val'] = multi.multipletests(statistics['P.Value'], method = 'fdr_bh')[1]
statistics['logFC'] = np.log2(statistics['FC'])
statistics = statistics.sort_values('P.Value')

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
statistics.to_csv('differential KOs.tsv', sep='\t')